# Домашнее задание

Домашнее задание состоит из нескольких блоков.


## Эксперименты в ipynb ноутбуках (15 баллов)
- Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ **(6 баллов)**
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(3 балла)**
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей. **(3 балла)**
- Придумать как можно обработать рекомендации для холодных пользователей. **(3 балла)**

Примечание: за невоспроизводимый код в ноутбуках (например, нарушен порядок выполнения ячеек, вызываются переменные, которые нигде не были объявлены ранее и.т.п) будут штрафы на усмотрение проверяющего.


## Реализация итоговой модели в сервисе (10 баллов)
- Пробитие бейзлайна $MAP@10 \geq 0.074921$ **(6 баллов)**
- Код сервиса соответствует критериям читаемости и воспроизводимости **(4 балла)**





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install implicit -q
!pip install rectools -q
!pip install lightfm -q
!pip install optuna -q

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import matplotlib.pyplot as plt
import typing as tp
import os
import warnings
import dill
import pickle
import nmslib
import time

from implicit.als import AlternatingLeastSquares
from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
from pathlib import Path
from tqdm import tqdm
from lightfm import LightFM
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization

In [4]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [5]:
warnings.filterwarnings('ignore')

In [6]:
DATA_PATH = '/content/drive/MyDrive/kion_train/'

# LOAD DATA 

In [7]:
%%time
users_df = pd.read_csv(DATA_PATH + 'users.csv')
items_df = pd.read_csv(DATA_PATH + 'items.csv')
interactions = pd.read_csv(DATA_PATH + 'interactions.csv')

CPU times: user 3.58 s, sys: 667 ms, total: 4.24 s
Wall time: 4.79 s


# Preprocess

In [8]:
Columns.Datetime = 'last_watch_dt'

In [9]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [10]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [11]:
max_date = interactions[Columns.Datetime].max()

In [12]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [13]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [14]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [15]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [16]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Prepare features

## User features

In [17]:
users_df.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [18]:
users_df.fillna('Unknown', inplace=True)

In [19]:
users_df.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [20]:
users = users_df.loc[users_df[Columns.User].isin(train[Columns.User])].copy()

In [21]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


In [22]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [23]:
user_features.query(f"id == 973171")

,id,value,feature
0,973171,М,sex
0,973171,age_25_34,age
0,973171,income_60_90,income


# Item features

In [24]:
items_df.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [25]:
items = items_df.loc[items_df[Columns.Item].isin(train[Columns.Item])].copy()

In [26]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [27]:
items.nunique()

item_id         14019
content_type        2
title           13454
title_orig       9724
release_year      104
genres           2559
countries         666
for_kids            2
age_rating          6
studios            38
directors        7414
actors          11830
description     13791
keywords        13583
dtype: int64

In [28]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [29]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [30]:
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


In [31]:
item_features = pd.concat((genre_feature, content_feature))

In [32]:
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


# Hyperparameters search(Optuna)

Подберем гиперпараметры для LightFM и ALS 

In [33]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_TRIALS = 15
LOSS = 'warp'

In [38]:
%%time
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: user 1.72 s, sys: 81.2 ms, total: 1.8 s
Wall time: 1.92 s


In [39]:
TEST_USERS = test[Columns.User].unique()

In [ ]:
def objective(trial):
    n_factors = trial.suggest_int('no_components', 16, 64, step=16)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log = True)
    
    lightfm = LightFMWrapperModel(
        LightFM(
            no_components=n_factors,
            loss=LOSS,
            learning_rate=lr,
            random_state=RANDOM_STATE
        ),
        num_threads=NUM_THREADS,
    )
    
    lightfm.fit(dataset)

    recos = lightfm.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    
    map_10 = MAP(k=K_RECOS).calc(recos, test)
    return map_10

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-11 14:54:26,818] A new study created in memory with name: no-name-6eb51dca-7046-40f9-bb83-a53b4ea65163
[I 2022-12-11 14:56:01,026] Trial 0 finished with value: 0.009432127264212135 and parameters: {'no_components': 32, 'learning_rate': 0.00013870722039573694}. Best is trial 0 with value: 0.009432127264212135.
[I 2022-12-11 14:57:30,592] Trial 1 finished with value: 0.0679174610090235 and parameters: {'no_components': 32, 'learning_rate': 0.0005553736722823529}. Best is trial 1 with value: 0.0679174610090235.
[I 2022-12-11 14:58:47,242] Trial 2 finished with value: 0.009357366662305697 and parameters: {'no_components': 16, 'learning_rate': 0.00019358437778645675}. Best is trial 1 with value: 0.0679174610090235.
[I 2022-12-11 15:00:01,740] Trial 3 finished with value: 0.0675838359279414 and parameters: {'no_components': 16, 'learning_rate': 0.00043558535255909373}. Best is trial 1 with value: 0.0679174610090235.
[I 2022-12-11 15:01:15,576] Trial 4 finished with value: 0.000442

Number of finished trials: 15
Best trial:
  Value: 0.07854116391924246
  Params: 
    no_components: 48
    learning_rate: 0.009507555095601732


In [ ]:
def objective(trial):
    n_factors = trial.suggest_int('no_components', 16, 64, step=16)
    reg = trial.suggest_float('learning_rate', 1e-4, 1e-1, log = True)
    
    als = ImplicitALSWrapperModel(
        AlternatingLeastSquares(
            factors=n_factors, 
            regularization=reg,
            random_state=RANDOM_STATE, 
            num_threads=NUM_THREADS,
        ),
        fit_features_together=True,
    )
    
    als.fit(dataset)

    recos = als.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    
    map_10 = MAP(k=K_RECOS).calc(recos, test)
    return map_10

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-11 15:19:31,164] A new study created in memory with name: no-name-2c87bc5e-c7be-42a1-af2a-f8c5823e9f9c
[I 2022-12-11 15:21:25,081] Trial 0 finished with value: 0.07460223241074236 and parameters: {'no_components': 64, 'learning_rate': 0.03542932017866923}. Best is trial 0 with value: 0.07460223241074236.
[I 2022-12-11 15:23:20,671] Trial 1 finished with value: 0.07466291097826479 and parameters: {'no_components': 64, 'learning_rate': 0.0014524363870082018}. Best is trial 1 with value: 0.07466291097826479.
[I 2022-12-11 15:25:03,235] Trial 2 finished with value: 0.07382740308329078 and parameters: {'no_components': 16, 'learning_rate': 0.025049831594355403}. Best is trial 1 with value: 0.07466291097826479.
[I 2022-12-11 15:26:56,887] Trial 3 finished with value: 0.07460267990822674 and parameters: {'no_components': 48, 'learning_rate': 0.0020432489819319027}. Best is trial 1 with value: 0.07466291097826479.
[I 2022-12-11 15:28:39,990] Trial 4 finished with value: 0.0740312613

Number of finished trials: 15
Best trial:
  Value: 0.07480028299600754
  Params: 
    no_components: 64
    learning_rate: 0.001109615409689709


Обучим лучшую модель на всех данных:

In [40]:
users_df = users_df.loc[users_df[Columns.User].isin(interactions[Columns.User])]

user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users_df.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)

In [41]:
items_df = items_df.loc[items_df[Columns.Item].isin(interactions[Columns.Item])]

items_df["genre"] = items_df["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items_df[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

content_feature = items_df.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))

In [42]:
full_dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [ ]:
best_model = LightFMWrapperModel(
    LightFM(
        no_components=48,
        loss=LOSS,
        learning_rate=0.0095,
        random_state=RANDOM_STATE
    ),
    num_threads=NUM_THREADS,
)
    
best_model.fit(full_dataset)

In [ ]:
with open('/content/drive/MyDrive/LightFM_best.dill', 'wb') as f:
    dill.dump(best_model, f)

In [44]:
with open('/content/drive/MyDrive/LightFM_best.dill', 'rb') as f:
    best_model = dill.load(f)

In [ ]:
recos = best_model.recommend(
    users=interactions['user_id'].unique(),
    dataset=full_dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [ ]:
recos = recos[['user_id', 'item_id']].groupby('user_id')['item_id'].apply(list)

In [ ]:
recos = recos.T.to_dict()

In [ ]:
with open('/content/drive/MyDrive/offline_lightfm.pkl', 'wb') as f:
    pickle.dump(recos, f)

# Approximate Nearest Neighbors 

In [47]:
user_embeddings, item_embeddings = best_model.get_vectors(full_dataset)

In [48]:
user_embeddings.shape, item_embeddings.shape

((962179, 50), (15706, 50))

In [49]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [50]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

pre shape:  (15706, 50)


(15706, 51)

In [51]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(962179, 51)

In [52]:
user_id = 30

In [53]:
user_embeddings[user_id]

array([-4.83524780e+01,  1.00000000e+00,  3.75187844e-01,  3.21771622e-01,
       -6.29868388e-01, -4.12927717e-01, -5.88016272e-01,  8.23162735e-01,
        3.88127029e-01, -1.18629479e+00,  2.73747325e-01, -3.02571565e-01,
       -7.79085159e-02, -4.71073091e-01, -5.45040727e-01, -5.40704846e-01,
       -4.42222714e-01,  3.65312696e-01,  3.23337018e-01,  6.10963106e-01,
        1.87580198e-01,  6.52648330e-01, -3.86717796e-01,  7.66267478e-01,
       -3.17292750e-01,  1.69645607e-01,  4.95087385e-01,  1.92529917e-01,
        6.13795817e-01,  4.07496661e-01,  1.57391593e-01, -2.01613009e-01,
        1.95904985e-01,  1.49788320e-01, -5.93021393e-01, -4.47738171e-01,
       -4.44614768e-01, -5.31978607e-01,  3.15818250e-01,  1.59003735e-01,
       -3.89377236e-01, -3.84812877e-02,  3.61991405e-01, -9.75922823e-01,
        2.45007321e-01,  2.57781893e-01,  8.22612226e-01, -9.48879346e-02,
       -8.41700435e-01, -6.60143137e-01])

In [54]:
augmented_user_embeddings[user_id]

array([-4.83524780e+01,  1.00000000e+00,  3.75187844e-01,  3.21771622e-01,
       -6.29868388e-01, -4.12927717e-01, -5.88016272e-01,  8.23162735e-01,
        3.88127029e-01, -1.18629479e+00,  2.73747325e-01, -3.02571565e-01,
       -7.79085159e-02, -4.71073091e-01, -5.45040727e-01, -5.40704846e-01,
       -4.42222714e-01,  3.65312696e-01,  3.23337018e-01,  6.10963106e-01,
        1.87580198e-01,  6.52648330e-01, -3.86717796e-01,  7.66267478e-01,
       -3.17292750e-01,  1.69645607e-01,  4.95087385e-01,  1.92529917e-01,
        6.13795817e-01,  4.07496661e-01,  1.57391593e-01, -2.01613009e-01,
        1.95904985e-01,  1.49788320e-01, -5.93021393e-01, -4.47738171e-01,
       -4.44614768e-01, -5.31978607e-01,  3.15818250e-01,  1.59003735e-01,
       -3.89377236e-01, -3.84812877e-02,  3.61991405e-01, -9.75922823e-01,
        2.45007321e-01,  2.57781893e-01,  8.22612226e-01, -9.48879346e-02,
       -8.41700435e-01, -6.60143137e-01,  0.00000000e+00])

In [55]:
item_id = 0

In [56]:
item_embeddings[item_id]

array([ 1.        ,  0.93062925,  0.04542188, -0.08912785, -0.40182877,
       -0.01549784, -0.11049332, -0.01765494,  0.05176789,  0.12865528,
        0.05936557, -0.20821114, -0.30196556,  0.05172248, -0.19765013,
        0.00188106, -0.04415952,  0.16934076,  0.30296585,  0.14377218,
        0.26332912,  0.04185844,  0.12309168,  0.24374816,  0.08689702,
        0.23112383,  0.0627045 , -0.09111952,  0.23607387,  0.20371312,
        0.17750743,  0.03330431,  0.46970484, -0.03104644, -0.16072789,
       -0.36810169,  0.04212458, -0.24139892,  0.17092629,  0.16394579,
       -0.22414672, -0.21966258,  0.16820855, -0.15324032,  0.00239269,
        0.07272961, -0.05856246, -0.20957781,  0.08415761, -0.1520945 ])

In [57]:
augmented_item_embeddings[item_id]

array([ 1.00000000e+00,  9.30629253e-01,  4.54218797e-02, -8.91278535e-02,
       -4.01828766e-01, -1.54978363e-02, -1.10493325e-01, -1.76549442e-02,
        5.17678931e-02,  1.28655285e-01,  5.93655668e-02, -2.08211139e-01,
       -3.01965564e-01,  5.17224781e-02, -1.97650135e-01,  1.88106298e-03,
       -4.41595241e-02,  1.69340760e-01,  3.02965850e-01,  1.43772185e-01,
        2.63329118e-01,  4.18584384e-02,  1.23091675e-01,  2.43748158e-01,
        8.68970156e-02,  2.31123835e-01,  6.27045035e-02, -9.11195204e-02,
        2.36073866e-01,  2.03713119e-01,  1.77507430e-01,  3.33043076e-02,
        4.69704837e-01, -3.10464352e-02, -1.60727888e-01, -3.68101686e-01,
        4.21245806e-02, -2.41398916e-01,  1.70926288e-01,  1.63945794e-01,
       -2.24146724e-01, -2.19662577e-01,  1.68208554e-01, -1.53240323e-01,
        2.39269063e-03,  7.27296099e-02, -5.85624613e-02, -2.09577814e-01,
        8.41576084e-02, -1.52094498e-01,  2.81623229e+00])

In [58]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}


In [59]:
# Number of neighbors 
K=10

In [60]:
# Space name should correspond to the space name 
# used for brute-force search
space_name='negdotprod'

In [61]:
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

15706

In [62]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 1.670063


In [63]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [64]:
# Querying
query_qty = augmented_user_embeddings.shape[0]
start = time.time() 
nbrs = index.knnQueryBatch(augmented_user_embeddings, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty)) 

kNN time total=36.979611 (sec), per query=0.000038 (sec), per query adjusted for thread number=0.000154 (sec)


In [65]:
nbrs[0]

(array([ 32,  16,  25,  84,  21, 235, 370, 174,  44, 173], dtype=int32),
 array([42.82581 , 42.887737, 43.208855, 43.479225, 43.63003 , 43.703815,
        43.70658 , 43.707195, 43.715534, 43.741734], dtype=float32))

In [66]:
nbrs[0][0]

array([ 32,  16,  25,  84,  21, 235, 370, 174,  44, 173], dtype=int32)

In [67]:
user_embeddings[[0], :]

array([[-4.79334106e+01,  1.00000000e+00,  5.16652584e-01,
         7.40993977e-01, -8.71812403e-02, -3.10741603e-01,
        -6.21468648e-02,  7.22927928e-01,  2.89485902e-01,
        -8.47943068e-01,  4.46044475e-01, -8.25431943e-01,
        -3.72506171e-01, -1.47162050e-01, -3.13372791e-01,
        -4.32358027e-01, -5.49459696e-01, -9.05489624e-02,
         2.49507278e-01,  8.46403986e-02,  7.10856199e-01,
         8.11529636e-01, -5.96860051e-01,  5.21905959e-01,
        -3.75617117e-01,  1.57306075e-01,  5.58442235e-01,
         6.01864278e-01,  5.07851362e-01,  5.26806295e-01,
         1.09351702e-01, -3.21162283e-01,  3.47558677e-01,
         5.74942112e-01, -7.18792379e-01, -2.14112476e-01,
        -6.48745418e-01, -1.17261544e-01,  4.55203384e-01,
         4.90635812e-01, -5.09907901e-01, -1.33621916e-01,
         6.18979812e-01, -4.83415127e-01, -1.61737874e-02,
         6.55006766e-02,  7.74052978e-01, -3.01507592e-01,
        -4.97673333e-01, -8.17249298e-01]])

In [68]:
all_users = interactions['user_id'].unique()

In [69]:
len(all_users)

962179

In [70]:
len(nbrs)

962179

In [71]:
recs = {all_users[i]: list(nbrs[i][0]) for i in range(len(nbrs))}

In [72]:
with open('/content/drive/MyDrive/offline_lightfm_ann.pkl', 'wb') as f:
    pickle.dump(recs, f)

# Solution for cold users

Будем просто рекомендовать холодным пользователям популярное

In [73]:
pop_model = PopularModel()
pop_model.fit(full_dataset)

In [74]:
pop_list = list(pop_model.recommend([0], full_dataset, k=300, filter_viewed=False)['item_id'][:10])

In [75]:
with open('/content/drive/MyDrive/popular.pkl', 'wb') as f:
    pickle.dump(pop_list, f)

# Avatars

Создадим 3 аватаров: любитель комедий, боевиков и ребенок, который смотрит мультфильмы

In [34]:
data = interactions.merge(items_df, on='item_id')

In [35]:
comedy_ids = list(data[data['genres'].isin(['комедии'])]['item_id'].value_counts()[:10].index)

In [36]:
comedy_ids

[4151, 3734, 4880, 11237, 7417, 1819, 12770, 8618, 13262, 16088]

In [38]:
items_df[items_df['item_id'].isin(comedy_ids)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
202,4880,series,Афера,Afera,2021.0,комедии,Россия,NaN,18.0,NaN,Михаил Старчак,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Смотри:- как кино- как сериалКарантин окончен,...","Афера, Аферисты, Карантин, Пандемия, Карантин ..."
4039,8618,film,Обратная связь,Obratnaya svyaz,2020.0,комедии,Россия,NaN,16.0,NaN,Алексей Нужный,"Леонид Барац, Ирина Горбачева, Ростислав Хаит,...",Новогодняя комедия от «Квартета И» и автора «Я...,"2020, россия, обратная, связь"
5726,13262,film,Папаши,È per il tuo bene,2020.0,комедии,Италия,NaN,16.0,NaN,Роландо Равелло,"Марко Джаллини, Винченцо Салемме, Джузеппе Бат...","Трое отчаявшихся отцов видят, что их худший ко...","2020, италия, папаши"
6425,11237,film,День города,NaN,2021.0,комедии,Россия,NaN,16.0,NaN,Алексей Харитонов,"Катерина Шпица, Антон Филипенко, Павел Ворожцо...",Эта история случилась в провинциальном городке...,"2021, россия, день, города"
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо..."
8537,7417,film,Стендап под прикрытием,Undercover standup,2020.0,комедии,Россия,NaN,16.0,NaN,Олег Асадулин,"Валентина Мазунина, Кирилл Нагиев, Зоя Бербер,...",Дерзкая и циничная опер в юбке Светлана Артюхо...,"2020, россия, стендап, под, прикрытием"
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"
13880,1819,film,Непосредственно Каха,Neposredstvenno Kakha,2020.0,комедии,Россия,NaN,16.0,NaN,Виктор Шамиров,"Артем Карокозян, Артем Калайджян, Марина Калец...",Южнороссийская хитовая комедия по мотивам попу...,"2020, россия, непосредственно, каха"
14379,12770,film,Соседи сверху,Sentimental,2020.0,комедии,Испания,NaN,18.0,NaN,Сеск Гай,"Белен Куэста, Хавьер Камара, Альберто Сан Хуан...",Хулио и Ана больше 15 лет вместе. Между ними д...,"2020, испания, соседи, сверху"
14661,16088,film,Дублёрша,The Stand In,2020.0,комедии,США,NaN,18.0,NaN,Джеми Бэббит,"Дрю Бэрримор, Майкл Зеген, ТиДжей Миллер, Холл...",Звезда комедий Кэнди Блэк ценит в этом мире ли...,"реабилитация, знаменитость, общественные работ..."


In [39]:
action_ids = list(data[data['genres'].isin(['боевики'])]['item_id'].value_counts()[:10].index)

In [40]:
action_ids

[12324, 9194, 991, 826, 5330, 5543, 13167, 6106, 11406, 12775]

In [41]:
items_df[items_df['item_id'].isin(action_ids)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
2463,826,film,Октагон: боец против рестлера,Cagefighter: Worlds Colided,2020.0,боевики,Великобритания,NaN,16.0,NaN,Джесси Кинонес,"Джина Гершон, Джонатан Гуд, Чак Лиделл, Джейсо...",Рисс – величайший чемпион ММА и легенда октаго...,"борьба, смешанные единоборства, 2020, соединен..."
2530,13167,film,Рейд 2,The Raid 2: Berandal,2014.0,боевики,"Индонезия, США",NaN,18.0,NaN,Гарет Эванс,"Ико Уайс, Арифин Путра, Тио Пакусодево, Ока Ан...","Из трех полицейских, выбравшихся из кровавого ...","тюрьма, под прикрытием, борьба, гангстер, поли..."
2818,5543,film,Турист,NaN,2021.0,боевики,Россия,NaN,16.0,NaN,Андрей Батов,"Алексей Шевченков, Евгений Терских, Александр ...",Декабрь 2020 года. Бывший сотрудник полиции Гр...,"2021, россия, турист"
5770,5330,film,Самый опасный человек,A Most Wanted Man,2014.0,боевики,"Великобритания, США, Германия",NaN,16.0,NaN,Антон Корбейн,"Григорий Добрыгин, Филип Сеймур Хоффман, Нина ...",Каждая война начинается со страха. Каждое пред...,"Мерседес-Бенц-сл, Форд, Форд Мондео, Альфа Ром..."
9090,9194,film,Роберт — король Шотландии,Robert the Bruce,2019.0,боевики,США,NaN,16.0,NaN,Ричард Грэй,"Энгус МакФадьен, Диармед Мёрта, Джаред Харрис,...","Для одних он - беглый преступник, дли других –...","Шотландия, семейные тайны, король, 14 век, отн..."
10907,6106,series,Ловушка,Lovushka,2013.0,боевики,Россия,NaN,12.0,NaN,Сергей Коротаев,"Иван Оганесян, Сергей Газаров, Игорь Ботвин, С...",Главе самого мощного городского мафиозного кла...,"зверство, криминал, мафия, полиция"
11972,12324,film,Патруль: По законам улиц,Shorta,2020.0,боевики,Дания,NaN,18.0,NaN,Фредерик Луис Хвиид,"Якоб Ульрик Ломанн, Саймон Сирс, Озлем Сагланм...",Полицейские Йенс и Майк регулярно патрулируют ...,"2020, дания, патруль, по, законам, улиц"
12872,12775,film,Мститель,Vengeance,2018.0,боевики,Великобритания,NaN,18.0,NaN,Росс Бояск,"Стю Беннет, Флер Кит, Алан Калтон, Гэри Дэниел...","Когда-то солдат, а теперь наёмник Джон Голд уз...","2018, соединенное королевство, мститель"
14044,11406,series,Буду жить,Budu zhit',2015.0,боевики,Россия,NaN,16.0,NaN,А.Коршунов,"Данила Якушев, Виолетта Яровая, Руслан Ягудин,...",Спустя 12 лет в свой родной приморский город в...,NaN
14952,991,film,Избави нас от лукавого,Daman akeseo guhasoseo,2020.0,боевики,Республика Корея,NaN,18.0,NaN,Хон Вон-чхан,"Хван Джон-мин, Ли Джон-джэ, Пак Чон-мин, Пак С...",Бывший секретный оперативник южнокорейских спе...,"бангкок, таиланд, месть, вкрутую, 2020, южная ..."


In [42]:
cartoon_ids = list(data[data['genres'].isin(['мультфильм'])]['item_id'].value_counts()[:10].index)

In [43]:
cartoon_ids

[11047, 599, 9950, 8134, 7844, 11578, 1264, 14683, 13471, 8237]

In [44]:
items_df[items_df['item_id'].isin(cartoon_ids)]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
1754,9950,film,Мама для мамонтенка,Mama dlya mamontyonka,1981.0,мультфильм,Россия,NaN,0.0,NaN,Олег Чуркин,"Зиновий Гердт, Зинаида Нарышкина, Клара Румяно...","Мультфильм о судьбе мамонтёнка, который случай...","1981, россия, мама, для, мамонтенка"
2390,11047,film,Гадкий утёнок,Gadkiy utyonok,2010.0,мультфильм,Россия,NaN,12.0,NaN,Гарри Бардин,"Светлана Степченко, Владимир Спиваков, Констан...",Странного вида утенок вылупляется из необычног...,", 2010, россия, гадкий, утёнок"
3436,7844,film,Крошка Енот,Kroshka Enot,1974.0,мультфильм,Россия,NaN,0.0,NaN,Олег Чуркин,"Клара Румянова, Мария Виноградова","Сегодня у Крошки Енота день рождения, и чтобы ...","1974, россия, крошка, енот"
3518,11578,film,Элька,Elka,2006.0,мультфильм,Россия,NaN,0.0,NaN,Владимир Саков,"Лариса Брохман, Алексей Булдаков, Дарья Белоус...",Эта история происходит накануне Нового года. Б...,"2006, россия, элька"
5651,1264,film,"Ишь ты, масленица","Ish ty, maslenitsa!",1985.0,мультфильм,Россия,NaN,6.0,NaN,Роберт Саакянц,"Ярослава Турылёва, Марина Гаврилко, Артем Кара...","О бедняке, наказавшем богача за жадность.","1985, россия, ишь, ты, масленица"
6590,8134,film,Большой Ух,Bolshoj Uh,1989.0,мультфильм,Россия,NaN,0.0,NaN,Юрий Бутырин,"Александр Пожаров, Зоя Пыльнова, Александр Ильин",Чудик с большими ушами слушал музыку звезд и н...,"1989, россия, большой, ух"
8361,599,film,Будь здоров!,Bud' zdorov!,2020.0,мультфильм,Кипр,NaN,0.0,NaN,Наумов Артем,Саранцева Варя,Маша подготовила несколько простых и о-очень п...,"2020, кипр, будь, здоров"
9785,13471,film,Адажио,Adazhio,2000.0,мультфильм,Россия,NaN,16.0,NaN,Гарри Бардин,NaN,"Философская притча, рассказывающая о серой тол...","2000, россия, адажио"
10741,8237,film,Воробьишка - хвастунишка,NaN,1981.0,мультфильм,Россия,NaN,0.0,NaN,Акмаль Акбарходжаев,NaN,В одном из живописных мест жили воробьи. Был с...,"1981, россия, воробьишка, хвастунишка"
12350,14683,film,Бабочки в животе,Des papillons dans le ventre,2020.0,мультфильм,Франция,NaN,0.0,NaN,Мириам Лазрак,NaN,"Мариус - скрупулезный бариста. Однажды, после ...","2020, франция, бабочки, животе"


Добавим информацию об аватарах

In [45]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [46]:
users_df.age.unique()

array(['age_25_34', 'age_18_24', 'age_45_54', 'age_35_44', 'Unknown',
       'age_55_64', 'age_65_inf'], dtype=object)

In [47]:
users_df.income.unique()

array(['income_60_90', 'income_20_40', 'income_40_60', 'income_0_20',
       'Unknown', 'income_90_150', 'income_150_inf'], dtype=object)

In [48]:
users_df['user_id'].max()

1097558

In [49]:
action_fan = {
    'user_id': 1100000,
    'age': 'age_25_34',
    'income': 'income_40_60',
    'sex': 'М',
    'kids_flg': 0
}

comedy_fan = {
    'user_id': 1100001,
    'age': 'age_18_24',
    'income': 'income_20_40',
    'sex': 'М',
    'kids_flg': 0
}

kid = {
    'user_id': 1100002,
    'age': 'Unknown',
    'income': 'Unknown',
    'sex': 'М',
    'kids_flg': 1
}

In [50]:
users_df = users_df.append(action_fan, ignore_index=True)
users_df = users_df.append(comedy_fan, ignore_index=True)
users_df = users_df.append(kid, ignore_index=True)

In [51]:
users_df.tail()

,user_id,age,income,sex,kids_flg
840195,590706,Unknown,Unknown,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0
840197,1100000,age_25_34,income_40_60,М,0
840198,1100001,age_18_24,income_20_40,М,0
840199,1100002,Unknown,Unknown,М,1


Добавим взаимодействия аватаров

In [52]:
interactions.tail()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
5476246,648596,12225,2021-08-13,76,0.0,1
5476247,546862,9673,2021-04-13,2308,49.0,3
5476248,697262,15297,2021-08-20,18307,63.0,3
5476249,384202,16197,2021-04-19,6203,100.0,3
5476250,319709,4436,2021-08-15,3921,45.0,3


In [53]:
interactions['last_watch_dt']

0         2021-05-11
1         2021-05-29
2         2021-05-09
3         2021-07-05
4         2021-04-30
             ...    
5476246   2021-08-13
5476247   2021-04-13
5476248   2021-08-20
5476249   2021-04-19
5476250   2021-08-15
Name: last_watch_dt, Length: 5476251, dtype: datetime64[ns]

In [54]:
d = {'user_id': [1100000, 1100000, 1100000, 1100000, 1100000, 1100000, 1100000, 1100000, 1100000, 1100000,
                 1100001, 1100001, 1100001, 1100001, 1100001, 1100001, 1100001, 1100001, 1100001, 1100001,
                 1100002, 1100002, 1100002, 1100002, 1100002, 1100002, 1100002, 1100002, 1100002, 1100002],
    'item_id': [*comedy_ids, *action_ids, *cartoon_ids],
    'last_watch_dt': [np.datetime64('2021-08-15') + np.timedelta64(i,'D') for i in range(30)],
    'total_dur': [7200 for _ in range(30)],
    'watched_pct': [100 for _ in range(30)],
    'weight': [3 for _ in range(30)]
}

In [55]:
for i in range(30):
    avtr_interaction = {}
    for key in d.keys():
        avtr_interaction[key] = d[key][i]
    interactions = interactions.append(avtr_interaction, ignore_index=True)

In [56]:
interactions.tail(15)

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
5476266,1100001,5543,2021-08-30,7200,100.0,3
5476267,1100001,13167,2021-08-31,7200,100.0,3
5476268,1100001,6106,2021-09-01,7200,100.0,3
5476269,1100001,11406,2021-09-02,7200,100.0,3
5476270,1100001,12775,2021-09-03,7200,100.0,3
5476271,1100002,11047,2021-09-04,7200,100.0,3
5476272,1100002,599,2021-09-05,7200,100.0,3
5476273,1100002,9950,2021-09-06,7200,100.0,3
5476274,1100002,8134,2021-09-07,7200,100.0,3
5476275,1100002,7844,2021-09-08,7200,100.0,3


Переобучим модель и посмотрим на рекомендации

In [57]:
users_df = users_df.loc[users_df[Columns.User].isin(interactions[Columns.User])]

user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users_df.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)

In [58]:
items_df = items_df.loc[items_df[Columns.Item].isin(interactions[Columns.Item])]

items_df["genre"] = items_df["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items_df[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

content_feature = items_df.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))

In [59]:
full_dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [60]:
best_model = LightFMWrapperModel(
    LightFM(
        no_components=48,
        loss=LOSS,
        learning_rate=0.0095,
        random_state=RANDOM_STATE
    ),
    num_threads=NUM_THREADS,
)
    
best_model.fit(full_dataset)

In [61]:
avtr_recs = best_model.recommend(
    users=[1100000, 1100001, 1100002],
    dataset=full_dataset,
    k=10,
    filter_viewed=True,
)

In [62]:
avtr_recs.merge(items_df, on='item_id').sort_values('user_id')

,user_id,item_id,score,rank,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
0,1100000,9728,-43.225472,1,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...","[боевики, триллеры]"
19,1100000,142,-44.023385,8,film,Маша,Masha,2020.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Анастасия Пальчикова,"Максим Суханов, Аня Чиповская, Полина Гухман, ...",13-летняя Маша растёт между боксёрским рингом ...,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма...","[драмы, триллеры]"
18,1100000,8636,-44.013938,7,film,Белый снег,Belyy sneg,2021.0,"драмы, спорт",Россия,NaN,6.0,NaN,Николай Хомерики,"Ольга Лерман, Федор Добронравов, Надежда Марки...",Отечественная спортивная драма про лыжницу и о...,"биография, занятие спортом, байопик, чемпионат...","[драмы, спорт]"
15,1100000,2657,-43.958969,6,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ...","[драмы, триллеры]"
12,1100000,6809,-43.842831,5,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни...",[документальное]
20,1100000,7571,-44.044031,9,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня...","[мультфильм, приключения, семейное, фэнтези, к..."
9,1100000,15297,-43.629754,4,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...","[драмы, мелодрамы]"
22,1100000,4740,-44.089664,10,film,Сахаров. Две жизни,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Иван Проскуряков,"Алексей Усольцев, Чулпан Хаматова, Агата Супер","Его имя носят проспект, университет и даже аст...","Сахаров, Сахарок, Сахар, Сахар сахаров, Сахаро...",[документальное]
6,1100000,13865,-43.331783,3,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...","[драмы, военные, приключения]"
3,1100000,10440,-43.314499,2,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...","[триллеры, детективы]"


Проскальзывают иногда релевантные рекомендации, но не релевантных достаточно много